In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# Read files

In [2]:
names = ["movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children's | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western | "]
names = [i.split(' | ') for i in names][0]
movies = pd.read_csv("u.item",delimiter="|",encoding="437",names=names)
movies['video release date'] = movies['release date']
ratings =pd.read_csv("u.data",delimiter="\t",names = ["user id","movie id","rating" ,"timestamp"])
u_cols = ['user id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols,
 encoding='latin-1')
n_users = users.shape[0]
movies.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,
0,1,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,NaN
1,2,GoldenEye (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,NaN
2,3,Four Rooms (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,NaN
3,4,Get Shorty (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,5,Copycat (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,NaN


# Data Cleaning 

In [3]:
ratings = ratings.rename(columns={'item id':'movie id'})
movies = movies.drop('',axis=1)

In [4]:
full_data = ratings.merge(movies,on='movie id',how="inner")
full_data.sort_values(by='movie id', inplace=True)
full_data.reset_index(inplace=True, drop=True)
full_data.head(1000)

,user id,movie id,rating,timestamp,movie title,release date,video release date,IMDb URL,unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,805,1,4,881695527,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
1,124,1,3,890287733,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
2,882,1,5,879864558,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
3,779,1,4,875501555,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
4,542,1,4,886532534,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
5,852,1,4,891036457,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
6,605,1,4,879365748,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
7,399,1,4,882340657,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
8,294,1,5,877819634,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
9,870,1,5,889717102,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
full_data.sort_values(by='movie id', inplace=True)
full_data.reset_index(inplace=True, drop=True)

In [6]:
full_data.head(1000)

,user id,movie id,rating,timestamp,movie title,release date,video release date,IMDb URL,unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,805,1,4,881695527,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
1,213,1,2,878870719,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
2,158,1,4,880132443,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
3,287,1,5,875334088,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
4,148,1,4,877019411,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
5,280,1,4,891700426,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
6,66,1,3,883601324,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
7,5,1,4,875635748,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
8,109,1,4,880563619,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0
9,181,1,3,878962392,Toy Story (1995),01-Jan-1995,01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
full_data[full_data.isnull().any(1)]

,user id,movie id,rating,timestamp,movie title,release date,video release date,IMDb URL,unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
40710,422,267,4,875655986,unknown,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
40711,130,267,5,875801239,unknown,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
40712,5,267,4,875635064,unknown,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
40713,833,267,1,875655669,unknown,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
40714,532,267,3,875441348,unknown,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
40715,1,267,4,875692955,unknown,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
40716,319,267,4,875707690,unknown,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
40717,297,267,3,875409139,unknown,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
40718,268,267,3,875742077,unknown,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
98803,181,1358,1,878962120,The Deadly Cure (1996),16-Sep-1996,16-Sep-1996,NaN,0,1,...,0,0,0,0,0,0,0,0,0,0


In [8]:
full_data = full_data.dropna(axis=0,subset=['release date','IMDb URL']).reset_index().drop('index',axis=1)


In [9]:
full_data[full_data.isnull().any(1)]

,user id,movie id,rating,timestamp,movie title,release date,video release date,IMDb URL,unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western


In [10]:
matrix= full_data.pivot_table(index='user id', columns='movie title', values='rating')
matrix.head()

movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Wyatt Earp (1994),Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),┴ k÷ldum klaka (Cold Fever) (1994)
user id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN


# Collaborative Filtering 

## Simple recommandation system based on Cosine and Pearson similarity

In [17]:
def CosineS(s1,s2):
    return np.sum(s1*s2)/np.sqrt(np.sum(s1**2)*np.sum(s2**2))
def pearsonR(s1, s2):
    s1_c = s1-s1.mean()
    s2_c= s2-s2.mean()
    return np.sum(s1_c*s2_c) / np.sqrt(np.sum(s1_c**2)* np.sum(s2_c**2))

def recommand(M=matrix,n=5):
    reviews=[]
    movie=input('Enter the movie you like: ')
    for title in M.columns:
        if title == movie:
            continue
        cor= pearsonR(M[movie], M[title])
        if np.isnan(cor):
            continue
        else:
            reviews.append((title,cor))
            
    reviews.sort(key= lambda tup: tup[1], reverse=True)
    return reviews[:n]

In [1]:
recommand()

NameError: name 'recommand' is not defined